program: Huffman Encoding

topic: Greedy / Compression

subtopic: Huffman Coding

source: GeeksForGeeks

url: [Huffman Coding | Greedy Algo-3](https://www.geeksforgeeks.org/dsa/huffman-coding-greedy-algo-3/)

difficulty: Medium

description: Given characters and their frequencies, build Huffman codes (prefix-free)
that assign shorter codes to more frequent characters. This notebook builds the tree,
prints codes, and draws the full Huffman tree with node labels (symbol/frequency) and
edge labels (0 for left, 1 for right).

Example taken from the GfG article: symbols = ['a','b','c','d','e','f'] with
freqs = [5,9,12,13,16,45]

In [ ]:
import heapq
import matplotlib.pyplot as plt

In [ ]:
class Node:
    def __init__(self, symbol=None, frequency=0):
        self.symbol = symbol
        self.frequency = frequency
        self.left = None
        self.right = None

    # make nodes comparable in heapq; break ties using id
    def __lt__(self, other):
        if self.frequency != other.frequency:
            return self.frequency < other.frequency
        return id(self) < id(other)

In [ ]:
def build_huffman_tree(symbols, freqs):
    """
    Build Huffman tree from given symbols and corresponding frequencies.
    Returns the root of the tree.
    """
    heap = []
    for sym, f in zip(symbols, freqs):
        heapq.heappush(heap, Node(sym, f))

    # corner case: single symbol -> return single node
    if len(heap) == 1:
        return heap[0]

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(None, left.frequency + right.frequency)
        merged.left = left
        merged.right = right
        heapq.heappush(heap, merged)

    return heap[0]

In [ ]:
def generate_codes(node, prefix="", codes=None):
    """
    Traverse the Huffman tree and generate codes (0 for left, 1 for right).
    For single-node tree, assign code '0'.
    """
    if codes is None:
        codes = {}

    if node is None:
        return codes

    # leaf node
    if node.symbol is not None:
        # single node special-case: give '0' at least one bit
        codes[node.symbol] = prefix if prefix != "" else "0"
        return codes

    generate_codes(node.left, prefix + "0", codes)
    generate_codes(node.right, prefix + "1", codes)

    return codes

In [ ]:
# Tree plotting helpers

def collect_edges(node, edges):
    """Collect edges as (parent_node, child_node, label) where label is '0' for left and '1' for right"""
    if node is None:
        return
    if node.left:
        edges.append((node, node.left, "0"))
        collect_edges(node.left, edges)
    if node.right:
        edges.append((node, node.right, "1"))
        collect_edges(node.right, edges)

def compute_positions(node, depth=0, positions=None, x_counter=None):
    """
    Compute (x,y) positions for each node using inorder traversal.
    positions: dict mapping node -> (x, y)
    x_counter: single-item list used as mutable integer reference
    """
    if positions is None:
        positions = {}
    if x_counter is None:
        x_counter = [0]

    if node.left:
        compute_positions(node.left, depth+1, positions, x_counter)

    # assign current node position
    positions[node] = (x_counter[0], -depth)
    x_counter[0] += 1

    if node.right:
        compute_positions(node.right, depth+1, positions, x_counter)

    return positions

def draw_huffman_tree(root, figsize=(6, 5)):
    """Draw the Huffman tree using matplotlib. Nodes labeled with symbol (if leaf) and frequency."""
    if root is None:
        print("Empty tree")
        return

    edges = []
    collect_edges(root, edges)
    positions = compute_positions(root)

    plt.figure(figsize=figsize)
    ax = plt.gca()
    ax.set_axis_off()

    # Draw edges with labels
    for parent, child, label in edges:
        x1, y1 = positions[parent]
        x2, y2 = positions[child]
        ax.plot([x1, x2], [y1, y2], color="gray", linewidth=1)
        # edge label at midpoint
        mx, my = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mx, my, label, fontsize=10, color="red", ha="center", va="center",
                bbox=dict(facecolor='white', edgecolor='none', pad=0.1, alpha=0.7))

    # Draw nodes
    for node, (x, y) in positions.items():
        # label: symbol if leaf, and frequency
        if node.symbol is not None:
            node_label = f"{node.symbol}\n{node.frequency}"
        else:
            node_label = f"{node.frequency}"
        circle = plt.Circle((x, y), 0.3, color='lightblue', ec='black')
        ax.add_patch(circle)
        ax.text(x, y, node_label, ha='center', va='center', fontsize=9)

    # autoscale
    xs = [p[0] for p in positions.values()]
    ys = [p[1] for p in positions.values()]
    margin = 1
    ax.set_xlim(min(xs) - margin, max(xs) + margin)
    ax.set_ylim(min(ys) - margin, max(ys) + margin)
    plt.title("Huffman Tree (label: symbol\\nfrequency; edge: 0-left / 1-right)")
    plt.show()

In [ ]:
if __name__ == "__main__":
    # Example from GfG:
    symbols = ['a', 'b', 'c', 'd', 'e', 'f']
    freqs = [5, 9, 12, 13, 16, 45]

    root = build_huffman_tree(symbols, freqs)
    codes = generate_codes(root)

    print("Character : Huffman Code")
    for sym in symbols:
        print(f"{sym:>3} : {codes[sym]}")

    # Draw the tree
    draw_huffman_tree(root, figsize=(8, 5))